# * **Listando todos os produtos, com seus titulos originais, traduzidos ou não para o português**
Lista de todos os produtos vendidos com seus respectivos nomes. Esses nomes podem estar traduzidos ou na linguagem natural do vendedor

In [ ]:
df = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')#
df

# * **Moedas utilizadas**
lista de tipo de moeda utilizada em cada venda

In [ ]:
df['currency_buyer']

# * **Preço dos produtos**
Lista de preço dos produtos vendidos e relação com moeda da venda

In [ ]:
df['price'] = df['currency_buyer']
print(df)

# * **Vendas Impulsionadas**
Se a venda foi impulsionada pelo vendedor, em caso de valor em 1, o produto foi impulsionado.

In [ ]:
df['uses_ad_boosts']

# * **Número de vendas ja realizadas**
Representa o total de vendas realizadas até o momento de cada produto

In [ ]:
df['units_sold']

# * **Avaliação média do produto**
Apresenta a avaliação média de cada produto

In [ ]:
df['rating']

# * **Número de avaliações recebida**
Número total de avaliações recebidas por cada produto

In [ ]:
df['rating_count']

# * **Número de avaliações 5 estrelas recebidas**
Número total de avaliações que o vendedor recebe dos clientes

In [ ]:
df['rating_five_count']

# * **Titulo original do produto**
Exibe os titulos originais de cada produto

In [ ]:
df['title']

# Final

A quick look at the products with 100000 sales.

In [ ]:
df[df['units_sold'] == 100000]

A simple function to create a new label 'is_successful':

In [ ]:
def is_successful(units_sold):
    if units_sold > 1000:
        return 1
    else:
        return 0

In [ ]:
df['is_successful'] = df['units_sold'].apply(is_successful)
#df['is_successful'] = df['units_sold'].apply(is_successful).astype('category')
print('Percent of successful products: ', df['is_successful'].value_counts()[1] / len(df['is_successful'])*100)
sns.countplot(data=df, x='is_successful')
plt.show()

I believe 33% is an appropriate % to be defined as successful sales. 

# Price and retail price and the relation of their difference..

I am not really sure what to get out of these fetures however from the histograms it looks like the retail price are much lower than the price at which the products are sold.

Maybe calculating the means might shade some light?

In [ ]:
print('Overall stats:')
print(df['price'].mean())
print(df['retail_price'].mean())
print('----------------------')
print('Stats for successful products:')
print(df[df['is_successful'] == 1]['price'].mean())
print(df[df['is_successful'] == 1]['retail_price'].mean())
print('----------------------')
print('Stats for unsuccessful products:')
print(df[df['is_successful'] == 0]['price'].mean())
print(df[df['is_successful'] == 0]['retail_price'].mean())

Nope.. Nothing speacial here..

One more trick I can use.. comparing the % change of price and retail price to the success of the products.

In [ ]:
df['diff_in_price'] = round(df['price']/df['retail_price'],2)
df['diff_in_price']

In [ ]:
sns.violinplot(data=df, y='diff_in_price', x='is_successful')

Nothing interesting here. I will just drop these columns if need be..

# The use of ad boosts to boost the success..

There is an almost equal use of ad boosts by the products.

In [ ]:
print('Percent of products using ad boosts: ', df['uses_ad_boosts'].value_counts()[1] / len(df['uses_ad_boosts'])*100)

In [ ]:
sns.countplot(data=df, x='uses_ad_boosts', hue='is_successful')

In [ ]:
pd.crosstab(df['uses_ad_boosts'], df['is_successful'])

In [ ]:
pd.crosstab(df['uses_ad_boosts'], df['units_sold'])

It seems like ad boosts do not have any impact on units sold.. perhaps something I am missing here? I mean, there must be some benefit of the addition cost(?) for the ad boosts. 

The website needs to consider the effectiveness of this, otherwise the merchants may simply not use it and the website might lose this additional income.

# Higher ratings means higher units sold?

First a look at the missing values.

In [ ]:
df[df['rating_five_count'].isnull()==True][['rating', 'rating_count',
       'rating_five_count', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count']]

The null values are all with 0 counts, but also have a rating of 5, which is strange. I will convert all these values to zero.

In [ ]:
def ratings_to_zero(rating_and_count):
    rating = rating_and_count[0]
    count = rating_and_count[1]
    
    if count == 0:
        rating = 0
    else:
        rating = rating
        
    return rating

In [ ]:
df['rating'] = df[['rating', 'rating_count']].apply(ratings_to_zero, axis=1)
df['rating_five_count'].fillna(0, inplace=True)
df['rating_four_count'].fillna(0, inplace=True)
df['rating_three_count'].fillna(0, inplace=True)
df['rating_two_count'].fillna(0, inplace=True)
df['rating_one_count'].fillna(0, inplace=True)


df[df['rating_five_count'].isnull()==True][['rating', 'rating_count',
       'rating_five_count', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count']]

In [ ]:
df['rating']

In [ ]:
ratings_column = ['rating', 'rating_count',
       'rating_five_count', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count']

for column in ratings_column:
    g = sns.FacetGrid(df, row='is_successful', margin_titles=True, aspect=4, height=3)
    g.map(plt.hist, column, bins=100)
    plt.title(column)
    plt.show()

In [ ]:
df.groupby('is_successful').mean()[ratings_column]

In [ ]:
df.groupby('units_sold').mean()[ratings_column]

Successful products have more ratings. This is expected as units sold is higher so rating is also higher. However, if I am building a model, I don't think it would be a good idea to keep this column as the huge number of ratings must be the result of higher units sold. 

# Badges 

In [ ]:
badges_column = ['badges_count',
       'badge_local_product', 'badge_product_quality', 'badge_fast_shipping']

In [ ]:
df[df['badges_count'] != 0][badges_column]

In [ ]:
for column in badges_column:
    sns.countplot(data=df, x=column, hue='is_successful')
    plt.title(column)
    plt.show()

Almost 50% of those who are successful have badges, especially the product quality badge. I am somwhat in a debate about whether I should use this as a feature or not, because does badges result in success or success result in badges? For now, I will assume badges results in sucess, since there are many successful products which do not have badges as well.

# Using tags to predict success

In [ ]:
from wordcloud import WordCloud, STOPWORDS

df['tags']

In [ ]:
def remove_stopwords(text):
    from nltk.tokenize import word_tokenize

    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in STOPWORDS]
    
    filtered_sentence = (" ").join(tokens_without_sw)

    return filtered_sentence

In [ ]:
df['tags'] = df['tags'].apply(remove_stopwords)

In [ ]:
df['tags']

In [ ]:
from collections import Counter
results = Counter()
df['tags'].str.lower().str.split().apply(results.update)
counter_df = pd.DataFrame.from_dict(results, orient='index')
counter_df.sort_values(by=0, axis=0, ascending=False).head(15)

In [ ]:
import matplotlib.pyplot as plt
word_string=" ".join(df['tags'].str.lower())
wordcloud = WordCloud(stopwords=STOPWORDS).generate(word_string)

plt.subplots(figsize=(15,15))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Most of the tags seems to be related to female's clothing. How about be compare the tags of successful and unsuccessful products.

In [ ]:
word_string=" ".join(df[df['is_successful']==1]['tags'].str.lower())
wordcloud = WordCloud(stopwords=STOPWORDS).generate(word_string)

plt.subplots(figsize=(15,15))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
word_string=" ".join(df[df['is_successful']==0]['tags'].str.lower())
wordcloud = WordCloud(stopwords=STOPWORDS).generate(word_string)

plt.subplots(figsize=(15,15))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Just for fun, predicting success based on tags only:

In [ ]:
X = df['tags']
y = df['is_successful']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.33)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics

text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words=STOPWORDS)),
                         ('clf', LinearSVC(random_state=0)),
    ])

text_clf.fit(X_train, y_train)  

predictions = text_clf.predict(X_test)

print('-----------------------')
print(metrics.confusion_matrix(y_test,predictions))
print('-----------------------')
print(metrics.classification_report(y_test,predictions))

Apart for EDA purposes, my limited knowledge in converting textual features to more useful numeric features is letting me down here. One of the learning points I have to keep in mind.. 

A few more word clouds for colour and size variation columns:

In [ ]:
df['product_color'].fillna('missing', inplace=True)
word_string=" ".join(df['product_color'].str.lower())
wordcloud = WordCloud(stopwords=STOPWORDS).generate(word_string)

plt.subplots(figsize=(15,15))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
df['product_variation_size_id'].fillna('missing', inplace=True)
word_string=" ".join(df['product_variation_size_id'].str.lower())
wordcloud = WordCloud(stopwords=STOPWORDS).generate(word_string)

plt.subplots(figsize=(15,15))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Shiping

In [ ]:
shipping_column = ['shipping_option_name',
       'shipping_option_price', 'shipping_is_express', 'countries_shipped_to']

for column in shipping_column:
    sns.countplot(data=df, x=column, hue='is_successful')
    plt.title(column)
    plt.show()

Better to visualise the 'countries_shipped_to' column with a violin plot.

In [ ]:
sns.violinplot(data=df, y='countries_shipped_to', x='is_successful')

I can't see any effect of shipping info columns on the products success.

# Inventory levels

From the first plot made, it seemed that there is only one value here

In [ ]:
df['inventory_total'].value_counts()

Now, I am confused..

In [ ]:
df['product_variation_inventory'].value_counts()

Perhaps, 'product_variation_inventory' is the right inventory to use?

In [ ]:
sns.violinplot(data=df, y='product_variation_inventory', x='is_successful')

As expected, successful products tend to be high in stock, however it would be more useful to compare against the units sold.

In [ ]:
sns.violinplot(data=df, y='product_variation_inventory', x='units_sold')

From the above plot, I can conclude that the inventory levels are kept at par for most of the products. However, it still seems that for some of the productsinvetory levels are kept on a very lower side. Perhaps this items are not being sold any more? The crawl data is of August only, however I am expecting the sales to be overall sales. 

From the very high count of 50 units, I am guessing this is just a maximum display and for some products there might be higher tha 50 units available. Maybe this is also the reason why for 'inventory_total', most values are 50. This might actually be the correct feature to use, but the website only displays up to a maximum of 50 units available.

# Urgency and country of origins

In [ ]:
df['has_urgency_banner'].fillna(0, inplace=True)
df['urgency_text'].fillna(0, inplace=True)

In [ ]:
urgency_column = ['has_urgency_banner', 'urgency_text', 'origin_country']

for column in urgency_column:
    sns.countplot(data=df, x=column, hue='is_successful')
    plt.title(column)
    plt.show()

Nothing interesting here..

# Successful merchants

In [ ]:
merchant_columns = ['merchant_rating_count', 'merchant_rating',
       'merchant_id', 'merchant_has_profile_picture']


sns.countplot(data=df, x='merchant_has_profile_picture', hue='is_successful')

There is a higher chance of success if merchant has profile picture.

In [ ]:
sns.violinplot(data=df, y='merchant_rating', x='is_successful')

In [ ]:
sns.violinplot(data=df, y='merchant_rating', x='units_sold')

From the above plots, the higher units sold tend to have higher ratings. 

In [ ]:
sns.countplot(x=df['merchant_id'].value_counts())

# The model

From the analysis above, I don't think this data is suitable for building a model as there is very minimal relation I noticed between data features and success. Also most features are impacted by success and not the other way round. however, I want to try out the SelectKBest feature selection, so here goes nothing...

In [ ]:
df.drop(labels=['title', 'title_orig', 'currency_buyer', 'merchant_title', 'merchant_name', 
                'merchant_info_subtitle', 'merchant_id', 'merchant_profile_picture', 'product_url', 
                'product_picture', 'product_id', 'theme', 'crawl_month', 'rating', 'rating_count',
       'rating_five_count', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count', 'units_sold','badges_count',
       'badge_local_product','badge_fast_shipping', 'tags', 'product_color',
       'product_variation_size_id', 'shipping_option_name',
       'shipping_option_price', 'shipping_is_express', 'countries_shipped_to','has_urgency_banner', 'urgency_text',
       'origin_country'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
y = df['is_successful']
X = df[['price', 'retail_price', 'uses_ad_boosts', 'badge_product_quality',
       'product_variation_inventory',
       'inventory_total', 'merchant_rating_count', 'merchant_rating',
       'merchant_has_profile_picture','diff_in_price']]

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


models = [LogisticRegression(), DecisionTreeClassifier(), XGBClassifier(),  
          GradientBoostingClassifier(), KNeighborsClassifier(), RandomForestClassifier()]

for model in models:
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(model)
    print('---------------------------')
    print(metrics.classification_report(y_test,y_pred))
    print('')
    print('')

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

for k in range(1,11):
    
    print(k)
    print('---------------')
    y = df['is_successful']
    X = df[['price', 'retail_price', 'uses_ad_boosts', 'badge_product_quality',
       'product_variation_inventory',
       'inventory_total', 'merchant_rating_count', 'merchant_rating',
       'merchant_has_profile_picture','diff_in_price']]


    X = SelectKBest(chi2, k=k).fit_transform(X, y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


    models = [LogisticRegression(), DecisionTreeClassifier(), XGBClassifier(),  
          GradientBoostingClassifier(), KNeighborsClassifier(), RandomForestClassifier()]

    for model in models:
    
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
        print(model)
        print('---------------------------')
        print(metrics.accuracy_score(y_test,y_pred))
        print('')
        print('')


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

y = df['is_successful']
X = df[['price', 'retail_price', 'uses_ad_boosts', 'badge_product_quality',
       'product_variation_inventory',
       'inventory_total', 'merchant_rating_count', 'merchant_rating',
       'merchant_has_profile_picture','diff_in_price']]


X = SelectKBest(chi2, k=8).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


models = RandomForestClassifier()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(metrics.accuracy_score(y_test,y_pred))


# Some conclusions

- The site mainly sells female clothing.
- Higher units sold means higher rating count. 
- The use of ad boosts does not seen to have any effect on the units sold and the site may lose revenue from this ads.
- More detailed units sold and inventory levels would have been more helpful for analysis.
- Product quality badges seems to increase the success of the products. Perhaps reviewing more products will increase motivation for merchants to improve their product? Different levels of badges can be applied?
- The tags can be improved so that products can be categorised more specfically. This can be done by reducing the number of tags per product, so the mechants are forced to choose their tags more wisely.
- Majority of the products are black and white. This might have been defined wrongly by the merchants. If not the case, the merchants can be encoured to include more variation to these.

# Final words..

I am marking this as final.

Feel free to leave any comments, reviews, improvements or if you just want to bash me up and it will help me improve my skills even more.

If you have reached this far, thanks a lot for reading!

Thanks to all those who commented and upvoted for keeping me motivated to continue.

And a lot of special thanks to **Jeffrey Mvutu Mabilama** for availing this data set! No data set, no data analysis!